In [ ]:
%pip install numpy

In [ ]:
%pip install tensorflow

In [ ]:
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout
from tensorflow.keras.optimizers import Adam
import os

# Define paths
base_dir = 'D:\PKG - C-NMC 2019\C-NMC_training_data'
folds = ['fold_0', 'fold_1', 'fold_2']
categories = ['all', 'hem']

# Parameters
img_width, img_height = 150, 150
batch_size = 32
epochs = 10  # Adjust based on your dataset size and available compute

# Data Generators
datagen = ImageDataGenerator(rescale=1.0/255.0, validation_split=0.2)

def create_data_generators(base_dir, folds, categories, img_width, img_height, batch_size):
    train_generators = []
    val_generators = []
    for fold in folds:
        train_data_dir = os.path.join(base_dir, fold)
        train_generator = datagen.flow_from_directory(
            train_data_dir,
            target_size=(img_width, img_height),
            batch_size=batch_size,
            class_mode='binary',
            subset='training'
        )
        val_generator = datagen.flow_from_directory(
            train_data_dir,
            target_size=(img_width, img_height),
            batch_size=batch_size,
            class_mode='binary',
            subset='validation'
        )
        train_generators.append(train_generator)
        val_generators.append(val_generator)
    return train_generators, val_generators

train_generators, val_generators = create_data_generators(base_dir, folds, categories, img_width, img_height, batch_size)

# Define a simple CNN model
def create_model():
    model = Sequential([
        Conv2D(32, (3, 3), activation='relu', input_shape=(img_width, img_height, 3)),
        MaxPooling2D((2, 2)),
        Conv2D(64, (3, 3), activation='relu'),
        MaxPooling2D((2, 2)),
        Conv2D(128, (3, 3), activation='relu'),
        MaxPooling2D((2, 2)),
        Flatten(),
        Dense(512, activation='relu'),
        Dropout(0.5),
        Dense(1, activation='sigmoid')
    ])
    model.compile(optimizer=Adam(learning_rate=0.001), loss='binary_crossentropy', metrics=['accuracy'])
    return model

# Train the model with each fold
for i, (train_generator, val_generator) in enumerate(zip(train_generators, val_generators)):
    print(f"Training with fold {i+1}")
    model = create_model()
    model.fit(
        train_generator,
        steps_per_epoch=train_generator.samples // batch_size,
        validation_data=val_generator,
        validation_steps=val_generator.samples // batch_size,
        epochs=epochs
    )
    model.save(f'model_fold_{i+1}.h5')
    print(f"Model for fold {i+1} saved as model_fold_{i+1}.h5")

Converting to single format and training a single model for the same.

In [15]:
import os
import numpy as np
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout
from tensorflow.keras.optimizers import Adam
from sklearn.metrics import recall_score


In [16]:
# Define paths
base_dir = 'D:/PKG - C-NMC 2019/C-NMC_training_data'
folds = ['fold_0', 'fold_1', 'fold_2']
categories = ['all', 'hem']

# Parameters
img_width, img_height = 150, 150
batch_size = 32
epochs = 10


In [18]:
# Updated Function to create data generators
def create_data_generators(base_dir, folds, categories, img_width, img_height, batch_size, validation_split=0.1):
    train_generators = []
    val_generators = []
    
    for fold in folds:
        # Directory for current fold
        fold_dir = os.path.join(base_dir, fold)
        
        # Create separate train and validation generators for each fold
        train_datagen = ImageDataGenerator(
            rescale=1.0/255.0,
            rotation_range=20,
            width_shift_range=0.2,
            height_shift_range=0.2,
            shear_range=0.2,
            zoom_range=0.2,
            horizontal_flip=True,
            validation_split=validation_split
        )
        
        train_generator = train_datagen.flow_from_directory(
            fold_dir,
            target_size=(img_width, img_height),
            batch_size=batch_size,
            class_mode='binary',  # Assuming binary classification
            subset='training'
        )
        
        val_generator = train_datagen.flow_from_directory(
            fold_dir,
            target_size=(img_width, img_height),
            batch_size=batch_size,
            class_mode='binary',  # Assuming binary classification
            subset='validation'
        )
        
        # Append generators to lists
        train_generators.append(train_generator)
        val_generators.append(val_generator)
    
    return train_generators, val_generators

# Create data generators
train_generators, val_generators = create_data_generators(base_dir, folds, categories, img_width, img_height, batch_size)


Found 3175 images belonging to 2 classes.
Found 352 images belonging to 2 classes.
Found 3224 images belonging to 2 classes.
Found 357 images belonging to 2 classes.
Found 3199 images belonging to 2 classes.
Found 354 images belonging to 2 classes.


In [19]:
from tensorflow.keras import layers, models

# Define the AlexNet model
def build_alexnet():
    model = models.Sequential()
    model.add(layers.Conv2D(96, kernel_size=(11, 11), strides=4, activation='relu', input_shape=(img_width, img_height, 3)))
    model.add(layers.MaxPooling2D(pool_size=(3, 3), strides=2))
    model.add(layers.Conv2D(256, kernel_size=(5, 5), padding='same', activation='relu'))
    model.add(layers.MaxPooling2D(pool_size=(3, 3), strides=2))
    model.add(layers.Conv2D(384, kernel_size=(3, 3), padding='same', activation='relu'))
    model.add(layers.Conv2D(384, kernel_size=(3, 3), padding='same', activation='relu'))
    model.add(layers.Conv2D(256, kernel_size=(3, 3), padding='same', activation='relu'))
    model.add(layers.MaxPooling2D(pool_size=(3, 3), strides=2))
    model.add(layers.Flatten())
    model.add(layers.Dense(4096, activation='relu'))
    model.add(layers.Dropout(0.5))
    model.add(layers.Dense(4096, activation='relu'))
    model.add(layers.Dropout(0.5))
    model.add(layers.Dense(1, activation='sigmoid'))
    return model

# Build the AlexNet model
alexnet = build_alexnet()

# Compile the model
alexnet.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])


In [7]:
import tensorflow as tf
from tensorflow.keras import layers, models

# Define the AlexNet model
def build_alexnet():
    model = models.Sequential()
    model.add(layers.Conv2D(96, kernel_size=(11, 11), strides=4, activation='relu', input_shape=(128, 128, 3)))
    model.add(layers.MaxPooling2D(pool_size=(3, 3), strides=2))
    model.add(layers.Conv2D(256, kernel_size=(5, 5), padding='same', activation='relu'))
    model.add(layers.MaxPooling2D(pool_size=(3, 3), strides=2))
    model.add(layers.Conv2D(384, kernel_size=(3, 3), padding='same', activation='relu'))
    model.add(layers.Conv2D(384, kernel_size=(3, 3), padding='same', activation='relu'))
    model.add(layers.Conv2D(256, kernel_size=(3, 3), padding='same', activation='relu'))
    model.add(layers.MaxPooling2D(pool_size=(3, 3), strides=2))
    model.add(layers.Flatten())
    model.add(layers.Dense(4096, activation='relu'))
    model.add(layers.Dropout(0.5))
    model.add(layers.Dense(4096, activation='relu'))
    model.add(layers.Dropout(0.5))
    model.add(layers.Dense(1, activation='sigmoid'))
    return model

# Build the AlexNet model
alexnet = build_alexnet()

# Compile the model
alexnet.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])


In [20]:
histories = []  # To store history for each fold

for i in range(len(folds)):
    print(f"Training on fold {i}")
    
    # Get the generators for this fold
    train_generator = train_generators[i]
    val_generator = val_generators[i]
    
    # Train the model for this fold
    history = alexnet.fit(
        train_generator,
        steps_per_epoch=train_generator.samples // batch_size,
        epochs=epochs,
        validation_data=val_generator,
        validation_steps=val_generator.samples // batch_size
    )
    
    histories.append(history)  # Store the history for this fold


Training on fold 0
Epoch 1/10
99/99 [==============================] - 151s 1s/step - loss: 0.5850 - accuracy: 0.7140 - val_loss: 0.5328 - val_accuracy: 0.7386
Epoch 2/10
99/99 [==============================] - 140s 1s/step - loss: 0.5318 - accuracy: 0.7607 - val_loss: 0.4959 - val_accuracy: 0.7841
Epoch 3/10
99/99 [==============================] - 156s 2s/step - loss: 0.5143 - accuracy: 0.7684 - val_loss: 0.6960 - val_accuracy: 0.4972
Epoch 4/10
99/99 [==============================] - 154s 2s/step - loss: 0.5288 - accuracy: 0.7560 - val_loss: 0.5464 - val_accuracy: 0.7642
Epoch 5/10
99/99 [==============================] - 170s 2s/step - loss: 0.5222 - accuracy: 0.7687 - val_loss: 0.5034 - val_accuracy: 0.7898
Epoch 6/10
99/99 [==============================] - 164s 2s/step - loss: 0.5154 - accuracy: 0.7716 - val_loss: 0.5132 - val_accuracy: 0.7585
Epoch 7/10
99/99 [==============================] - 172s 2s/step - loss: 0.5078 - accuracy: 0.7677 - val_loss: 0.5276 - val_accuracy: 0

In [23]:
from sklearn.metrics import recall_score
import numpy as np

# Function to evaluate model and compute recall, accuracy, and loss
def evaluate_model(model, generator):
    # Get true labels
    y_true = generator.classes
    
    # Predict probabilities
    y_pred_prob = model.predict(generator)
    
    # Convert probabilities to binary predictions
    y_pred_binary = np.round(y_pred_prob).flatten()
    
    # Calculate recall
    recall = recall_score(y_true, y_pred_binary)
    
    # Calculate accuracy and loss
    loss, accuracy = model.evaluate(generator, verbose=0)
    
    return recall, accuracy, loss

# Evaluate on each fold
for i in range(len(folds)):
    print(f"Evaluating on fold {i}")
    val_generator = val_generators[i]
    recall, accuracy, loss = evaluate_model(alexnet, val_generator)
    print(f"Recall for fold {i}: {recall:.4f}")
    print(f"Accuracy for fold {i}: {accuracy:.4f}")
    print(f"Loss for fold {i}: {loss:.4f}")


Evaluating on fold 0
11/11 [==============================] - 10s 936ms/step
Recall for fold 0: 0.2389
Accuracy for fold 0: 0.7841
Loss for fold 0: 0.4987
Evaluating on fold 1
12/12 [==============================] - 10s 857ms/step
Recall for fold 1: 0.2586
Accuracy for fold 1: 0.8571
Loss for fold 1: 0.4479
Evaluating on fold 2
12/12 [==============================] - 9s 812ms/step
Recall for fold 2: 0.1651
Accuracy for fold 2: 0.8446
Loss for fold 2: 0.4488


In [22]:
# Save the model
alexnet.save('alexnet_binary.h5')
print("Model saved as alexnet_binary.h5")

Model saved as alexnet_binary.h5


In [ ]:
# Evaluation on test set (fold_1)
test_generator = datagen.flow_from_directory(
    os.path.join(base_dir, folds[1]),  # Use fold_1 for testing
    target_size=(img_width, img_height),
    batch_size=batch_size,
    class_mode='binary',  # Assuming binary classification
    subset='validation',  # Use 'validation' subset
    shuffle=False  # Keep data order to match predictions with labels
)

# Evaluate the model
loss, accuracy = model.evaluate(test_generator)

# Predict probabilities for the test set
y_pred_prob = model.predict(test_generator)

# Convert probabilities to class labels
y_pred = (y_pred_prob > 0.5).astype(int)

# Extract true labels
y_true = test_generator.classes

# Calculate recall (sensitivity) using sklearn's recall_score
from sklearn.metrics import recall_score

recall = recall_score(y_true, y_pred)

print(f"Recall: {recall:.4f}")
print(f"Test Loss: {loss:.4f}")
print(f"Test Accuracy: {accuracy:.4f}")


Completed the entire process with purely one folder

In [ ]:

import os
import cv2
import numpy as np
from tensorflow.keras.utils import to_categorical
from PIL import Image
from skimage.util import view_as_blocks

def load_test_data(folder, image_patch_size):
    image_dataset = []
    labels = []

    for file_name in os.listdir(folder):
        if file_name.endswith('.bmp'):
            image_path = os.path.join(folder, file_name)
            print(f'Loading test image: {image_path}')
            image = cv2.imread(image_path, 1)
            if image is not None:
                size_x = (image.shape[1] // image_patch_size) * image_patch_size
                size_y = (image.shape[0] // image_patch_size) * image_patch_size
                image = Image.fromarray(image)
                image = image.crop((0, 0, size_x, size_y))
                image = np.array(image)
                patched_images = view_as_blocks(image, (image_patch_size, image_patch_size, 3))
                for i in range(patched_images.shape[0]):
                    for j in range(patched_images.shape[1]):
                        individual_patched_image = patched_images[i, j, :, :]
                        image_dataset.append(individual_patched_image)
                        labels.append(0)  # Assuming all images are of the same class (label 0)
            else:
                print(f'Failed to load test image: {image_path}')

    image_dataset = np.array(image_dataset)
    labels = np.array(labels)
    labels = to_categorical(labels, 2)  # Convert labels to categorical if needed

    return image_dataset, labels

# Define your test folders
test_folder1 = "D:\PKG - C-NMC 2019\C-NMC_test_prelim_phase_data"
test_folder2 = "D:\PKG - C-NMC 2019\C-NMC_test_final_phase_data"

# Assuming image_patch_size is defined elsewhere in your code
image_patch_size = 128

# Load test data from both folders
test_image_dataset1, test_labels1 = load_test_data(test_folder1, image_patch_size)
test_image_dataset2, test_labels2 = load_test_data(test_folder2, image_patch_size)

In [ ]:
import numpy as np
from tensorflow.keras.preprocessing.image import array_to_img

# Function to preprocess test data
def preprocess_test_data(test_data, target_size=(150, 150)):
    preprocessed_data = []
    
    for batch in test_data:
        batch_preprocessed = []
        for patch in batch:
            img = array_to_img(patch.astype('uint8'))  # Convert to PIL Image
            img = img.resize(target_size)
            image = np.array(img)
            batch_preprocessed.append(image)
        
        preprocessed_data.append(np.array(batch_preprocessed))
        
    return np.array(preprocessed_data)

# Assuming image_patch_size is defined elsewhere in your code
image_patch_size = 128

# Preprocess test datasets
test_image_dataset1_processed = preprocess_test_data(test_image_dataset1)
test_image_dataset2_processed = preprocess_test_data(test_image_dataset2)


In [ ]:
import tensorflow as tf
from tensorflow.keras.models import load_model
import numpy as np

# Define a function to evaluate the model
def evaluate_model(model_path, test_data, test_labels):
    model = load_model(model_path)
    # Evaluate the model
    loss, accuracy = model.evaluate(test_data, test_labels, verbose=1)
    print(f'Model evaluation - Loss: {loss}, Accuracy: {accuracy}')

# Assuming paths to your trained model files
model_path_fold1 = 'model_fold_1.h5'
model_path_fold2 = 'model_fold_2.h5'

# Reshape test_labels to match model output shape (None, 1)
test_labels1_single = test_labels1[:, 1]  # Assuming class 'hem' is encoded as 1
test_labels2_single = test_labels2[:, 1]  # Assuming class 'hem' is encoded as 1

# Evaluate model on fold 1
evaluate_model(model_path_fold1, np.squeeze(test_image_dataset1_processed), test_labels1_single)

# Evaluate model on fold 2
evaluate_model(model_path_fold2, np.squeeze(test_image_dataset2_processed), test_labels2_single)


In [ ]:
import numpy as np
from tensorflow.keras.models import load_model
from sklearn.metrics import recall_score
from tensorflow.keras.preprocessing.image import array_to_img
import os
import cv2
from tensorflow.keras.utils import to_categorical
from PIL import Image
from skimage.util import view_as_blocks

# Function to calculate recall
def calculate_recall(model_path, test_data, test_labels):
    # Load the model
    model = load_model(model_path)
    
    # Make predictions
    predictions = model.predict(test_data)
    predicted_classes = np.argmax(predictions, axis=1)
    
    # Check if there are any positive samples in the test labels
    print(f'Number of positive samples in test labels: {np.sum(test_labels == 1)}')
    
    # Calculate recall
    recall = recall_score(test_labels, predicted_classes, zero_division=1)
    print(f'Recall: {recall:.2%}')
    
    return recall

# Paths to your trained model files
model_path_fold1 = 'model_fold_1.h5'
model_path_fold2 = 'model_fold_2.h5'

# Calculate recall for fold 1
recall_fold1 = calculate_recall(model_path_fold1, np.squeeze(test_image_dataset1_processed), test_labels1_single)

# Calculate recall for fold 2
recall_fold2 = calculate_recall(model_path_fold2, np.squeeze(test_image_dataset2_processed), test_labels2_single)
